# Portfolio Optimisation AI

Portfolio Optimisation AI uses deep reinforcement learning to actively control A portfolio of input stocks, cryptocurrencies to optimize portfolio risk and return. Deep reinforcment learning is used to maximise porfolio's logarithmic return by 'learning' to mitigate risk and improve portfolio sortano and sharpe ratios.  



# Methods

G is global array of previous timeseries data of m selected stocks and is used as the input data for the model. First row contains the 'risk free' asset and hence has constant value 1. all other timeseries price data is normalised to the price of rick free asset at that time. 
 $$ G =
  \left[ {\begin{array}{cccc}
    1 & 1 & \cdots & 1\\
    x_{21} & x_{22} & \cdots & x_{2T}\\
    \vdots & \vdots & \ddots & \vdots\\
    x_{m1} & x_{m2} & \cdots & x_{mT}\\
  \end{array} } \right]$$
The proportion of capital invested in each stock is contained in vector w_t.

The states of the envirment $S_t$ used as input into networks is given by: 
 $$ S_t =
  \left[ {\begin{array}{cccc}
    1 & 1 & \cdots & 1\\
    \frac{x_{2,t-w+1}}{x_{2,t}} & \frac{x_{2,t-w+1}}{x_{2,t}} & \cdots & 1\\
    \vdots & \vdots & \ddots & \vdots\\
   \frac{x_{m,t-w+1}}{x_{m,t}} & \frac{x_{2,t-w+1}}{x_{m,t}} & \cdots & 1\\
  \end{array} } \right]$$
  
The return of stock in a portfolio is defined by the ratio of the closing price over the opening price of the stock for given trading peroid. This can be calculated by 
$$w_t\cdot y_t$$ where y_t is price change vector overperiod t ,
$$y_t = \left[\begin{array}{cccc}\frac{x_{1,t}}{x_{1,t-1}} &\frac{x_{2,t}}{x_{2,t-1}}& \cdots &\frac{x_{m,t}}{x_{m,t-1}}\\\end{array}\right] $$
The return of portfolio after some time T given by: 
$$ R_t = \prod_{t=1}^Tr_t(1-c\sum_{i=1}^m|w_{t,i}-w_{t-1,i}|)$$

The reward use by the algorithm is calulated as:
$$ \sum_{t=1}^T\ln\mu_tw_t\cdot y_t  $$
where $\mu$ is the factor of which of which portfolio shrinks by due to transaction costs. 
$\mu $ is calculated iteratively by:
$$ \mu_t = \frac{1}{1-C_pw_{t,0}}\left(1-C_pw_{0,t}-C_s+C_p-C_sC_p)\sum_1^m(w_{t,i}-\mu_t{w_t,i})^+   \right)  $$




***
$\mathbf{\text{Training Algorithm: 'On policy' PPO style }}$<br>
***
1.&emsp;Initialize policy network $\mu$ with weights $\theta^{\mu}$ and Critic network $Q(S_t,a|\theta^{\mu})$ with weights $\theta^{Q}$ and trajectory buffer $R_n$.

2.&emsp;For t = 1 to T:<br>
  &emsp;&emsp;If $\mod{t} = Size(R_n)$
  
&emsp;&emsp;(a)&emsp;Use policy network to produce the action mean $ \textbf{m}(S_t)\in \mathbb{R}^n$ and standar deviation $ \boldsymbol{\sigma}(S_t) \in \mathbb{R}^{n}$

&emsp;&emsp;(b) execute action $a_t \sim \mathcal{N}(\textbf{m},\,\boldsymbol{\sigma})$
calculate $p_t = \ln(P(a_t)) $ and $Q(a_t,S_t)$

&emsp;&emsp;(c) Solve the optimization problem (1) for the one step greedy
action $\bar{a}$

&emsp;&emsp;(d) Store trajectory $(S_t,a_t,r_t,p_t,Q_t)$ in $R_n$

3.&emsp; Sample random batches from  $R_n$

  &emsp; calculate probabitlity ratio for batch \$$R(\theta^\mu)= \frac{\mu_{\theta^{\mu}}(S_t,a_t)}{ \mu_{\theta^{\mu} old}(S_t,a_t)} $$
  
  &emsp;&emsp;estimate value at t  $Q(s_t,a_t)$ and calulate the advantage $$\hat{A}_t = \sum_{\tau>t}\gamma^{\tau-t}\lambda_\tau - Q(s_t,a_t)$$ 
  
  calulate loss $$\boldsymbol{\nabla}_{\theta^\mu}\frac{1}{B}\sum_{t=1}^N\sum_{b=1}^B \min\left(\frac{\mu_{\theta^{\mu}}(S_t,a_t)}{ \mu_{\theta^{\mu} old}(S_t,a_t)},clip\left(\frac{\mu_{\theta^{\mu}}(S_t,a_t)}{ \mu_{\theta^{\mu} old}(S_t,a_t)},1-\eta,1+\eta\right)\right)$$
  
  calculate loss $$\boldsymbol{\nabla}_{\theta^\mu}\frac{1}{B}\sum_{t=1}^N \hat{A}^2$$